In [1]:
!pip install transformers[sentencepiece]

In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 KB 288.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=57e1c4d5ae2d3d540cb52c86116d65576aa87f39d67676163b706f1b90a9b454
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [3]:
from sentence_transformers import SentenceTransformer, util, models
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import pandas as pd
import spacy
import numpy as np
from scipy.spatial.distance import cosine

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/sentence-t5-base')
model = AutoModel.from_pretrained('sentence-transformers/sentence-t5-base')
review = pd.read_csv("../input/dataset-of-df/df_app_reviews.csv")
iss = pd.read_csv("../input/dataset-of-df/df_app_issue.csv",encoding="ISO-8859-1")

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/209M [00:00<?, ?B/s]

Some weights of T5Model were not initialized from the model checkpoint at sentence-transformers/sentence-t5-base and are newly initialized: ['decoder.block.4.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.11.layer.0.SelfAttention.v.weight', 'decoder.block.1.layer.1.EncDecAttention.k.weight', 'decoder.block.4.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.1.layer_norm.weight', 'decoder.block.8.layer.2.DenseReluDense.wi.weight', 'decoder.block.10.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer.2.layer_norm.weight', 'decoder.block.10.layer.2.DenseReluDense.wi.weight', 'decoder.block.10.layer.2.layer_norm.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.5.layer.1.EncDecAttention.k.weight', 'decoder.block.11.layer.1.EncDecAttention.q.weight', 'decoder.block.5.layer.2.layer_norm.weight', 'decoder.bloc

In [4]:
feedback = review.vlc.unique()
issue = iss.vcl.unique()

In [5]:
dic = np.load('../input/nounvectorstt5vlc/noun-vector-stTt5-vcl.npy', allow_pickle='TRUE')
# print(dic.item().keys())
new_dic = dic.item() #将nmupy.ndarray格式转换为dict
print(type(new_dic))

<class 'dict'>


In [6]:
import torch.nn as nn
class ConineSimilarity(nn.Module):

    def forward(self, tensor_1,tensor_2):
        normalized_tensor_1=tensor_1/tensor_1.norm(dim=-1,keepdim=True)
        normalized_tensor_2=tensor_2/tensor_2.norm(dim=-1,keepdim=True)
        return (normalized_tensor_1*normalized_tensor_2).sum(dim=-1)

con = ConineSimilarity()

In [7]:
def getvec(words):
  input_ids = tokenizer(words, padding=True, truncation=True, return_tensors='pt').input_ids
  with torch.no_grad():
    outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)
  last_hidden_states =  outputs.last_hidden_state 
  pooled_sentence = torch.mean(last_hidden_states, dim=1)

  return pooled_sentence

In [8]:
def search_key(words):
    
    vec = getvec(words)
    # pingjun1 = pingjun(words)
    CS1 = con(vec,list(new_dic.values())[0])  #将字典转换为列表，方便索引
    CS2 = con(vec,list(new_dic.values())[1])
    CS3 = con(vec,list(new_dic.values())[2])
    Iss1 = list(new_dic.values())[0]
    Iss2 = list(new_dic.values())[1]
    Iss3 = list(new_dic.values())[2]
#     print(CS1,CS2,CS3)
    max1 = max2 = max3=0
    #先将前三个的相似度按顺序排好为max1,max2,max3
    if(CS1>=CS2 and CS1>=CS3):
      max1 = CS1
      Iss1 = issue[0]
      if(CS2 >= CS3):
        max2 = CS2
        Iss2 = issue[1]
        max3 = CS3
        Iss3 = issue[2]
      else:
        max2 = CS3
        Iss2 = issue[2]
        max3 = CS2
        Iss3 = issue[1]
    elif(CS2>=CS3 and CS2>=CS1):
      max1 = CS2
      Iss1 = issue[1]
      if (CS3 >= CS1):
        max2 = CS3
        Iss2 = issue[2]
        max3 = CS1
        Iss3 = issue[0]
      else:
        max2 = CS1
        Iss2 = issue[0]
        max3 = CS3
        Iss3 = issue[2]
    elif(CS3>=CS2 and CS3>=CS1):
      max1 = CS3
      Iss1 = issue[2]
      if (CS2 >= CS1):
        max2 = CS2
        Iss2 = issue[1]
        max3 = CS1
        Iss3 = issue[0]
      else:
        max2 = CS1
        Iss2 = issue[0]
        max3 = CS2
        Iss3 = issue[1]
    i=3 #计数器，从第三条开始计算
        # 在比较剩下的和max1,max2,max3的大小
    for iss in list(new_dic.values())[3:]:
#             issvec = dic[iss]
            CS = con(vec,iss)
            if(CS > max3 and CS < max2):
                max3 = CS
                Iss3 = issue[i]
            elif(CS >max3 and CS > max2 and CS < max1):
                max3 = max2
                Iss3 = Iss2
                max2 = CS
                Iss2 = issue[i]
            elif(CS > max1):
                max3 = max2
                Iss3 = Iss2
                max2 = max1
                Iss2 = Iss1
                max1 = CS
                Iss1 = issue[i]

            i=i+1

#     print(max1,Iss1,max2,Iss2,max3,Iss3)
    return words,max1,Iss1,max2,Iss2,max3,Iss3
    
# vector_data = pd.DataFrame({"issue_text":issue_text,"vector":v})
# vector_data.to_csv("nextcloud_vec.csv",index=False)


In [9]:
words,max1,Iss1,max2,Iss2,max3,Iss3 =search_key("App works only few hours, next - problems with login. Can't login, must reinstall app again, and again!")
f = []
i = []
d = []

f.append(words)
f.append(words)
f.append(words)
i.append(Iss1)
i.append(Iss2)
i.append(Iss3)
d.append(max1.detach().numpy()[0])
d.append(max2.detach().numpy()[0])
d.append(max3.detach().numpy()[0])

print(f)
print(i)
print(d)

["App works only few hours, next - problems with login. Can't login, must reinstall app again, and again!", "App works only few hours, next - problems with login. Can't login, must reinstall app again, and again!", "App works only few hours, next - problems with login. Can't login, must reinstall app again, and again!"]
['Do not display last directory in hierarchy whith no media, like "Covers", "Lirycs" e.t.c. in "Directories" view.', 'hdr10 4k stutters, then freezes on android (nokia 7.1 android 8.1)', 'Unable to fast forward or fast rewind in a *.ts file at playback']
[0.612063, 0.60242116, 0.58654505]


In [10]:
f = []
i = []
d = []

feedback = review.vlc.unique()
issue = iss.vcl.unique()

for fb1 in feedback:

    words,max1,Iss1,max2,Iss2,max3,Iss3 =search_key(fb1)
    f.append(words)
    f.append(words)
    f.append(words)
    i.append(Iss1)
    i.append(Iss2)
    i.append(Iss3)
    d.append(max1.detach().numpy()[0])
    d.append(max2.detach().numpy()[0])
    d.append(max3.detach().numpy()[0])

In [11]:
data = pd.DataFrame({'feedback':f,'issue':i,'distance':d})
data.to_csv("vcl_stT5_noun.csv",index=False)